In [111]:
import json 
import os
import logging 
import concurrent 
import asyncio 
import re 
import requests 

from collections import deque 
from itertools import chain 
# from datetime import date, datetime 

from typing import List 
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from palettable.tableau import Tableau_20
from palettable.mycarta import Cube1_4, Cube1_8
from subgrounds.pagination import ShallowStrategy

from web3 import Web3
from concurrent.futures import ThreadPoolExecutor

# apis / networking 
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport
from etherscan import Etherscan
from pycoingecko import CoinGeckoAPI

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss
from IPython.display import HTML, display


from utils import (
    ddf, remove_prefix, query_attrs, camel_to_snake, df_cols_camel_to_snake, df_cols_change_prefix
)

# logging.basicConfig(level=logging.INFO)

pp = PrettyPrinter().pprint

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [60]:
colors_24 = Tableau_20.hex_colors + Cube1_4.hex_colors
colors_28 = Tableau_20.hex_colors + Cube1_8.hex_colors

In [61]:
URL_INFURA = 'https://mainnet.infura.io/v3/856c3834f317452a82e25bb06e04de18'
w3 = Web3(Web3.HTTPProvider(URL_INFURA))
cg = CoinGeckoAPI()
hmmmm = 'VR7YA9DRDB4Y15B5N3WU9E7PSJ9RWPCP5S'
etherscan = Etherscan(hmmmm)

In [62]:
URL_CURVE_POOLS = 'https://api.thegraph.com/subgraphs/name/convex-community/curve-pools'
URL_CURVE_DAO = 'https://api.thegraph.com/subgraphs/name/convex-community/curve-dao'

ADDRESS_frxETHCRV = '0xf43211935c781d5ca1a41d2041f397b8a7366c7a'.lower()
ADDRESS_frxETHCRV_GAUGE_DEPOSIT = '0x2932a86df44fe8d2a706d8e9c5d51c24883423f5'.lower()
ADDRESS_CONVEX_VOTER_PROXY = '0x989aeb4d175e16225e39e87d0d97a3360524ad80'.lower()
ADDRESS_CURVE_POOL_FRXETH_ETH = '0xa1f8a6807c402e4a15ef4eba36528a3fed24e577'.lower()

# Analysis of Holders of frxETHCRV 

[frxETHCRV](https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a) is the liquidity pool token for the [frxETH:ETH pool](https://curve.fi/#/ethereum/pools/frxeth/deposit) on curve. 

Tracking which users hold frxETHCRV is not as simple as checking the total amount of this token across user accounts. 
There are various platforms where users can stake frxETHCRV in exchange for derivative tokens. This hierarchical outline
shows the various places where the supply of frxETHCRV exists within the ecosystem. 

- frxETHCRV supply 
    - circulating frxETHCRV *(ignored for now since ~99.9% of frxETHCRV is not circulating, see [here](https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a#balances))*. 
    - staked in [curve gauge](0x2932a86df44fe8d2a706d8e9c5d51c24883423f5) for frxETH:ETH pool. 
        - User stakes directly in curve gauge (frxETHCRV -> frxETHCRV-gauge). 
        - User uses convex staking to indirectly stake into curve gauge. 
            - Convex Curve Staking (frxETHCRV -> cvxfrxETHCRV (user receives this) -> frxETHCRV-gauge (convex holds internally))
            - Convex Frax Staking (frxETHCRV -> stkcvxfrxETHCRV-frax)
                - Vaulted Staking (earning boosted yield in FXS through convex) 
                - Non-Vaulted Staking (Why would you do this?) 


In [80]:
def get_account_balance_of_erc20(account, token_address): 
    account = Web3.toChecksumAddress(account) 
    token_address = Web3.toChecksumAddress(token_address)
    abi = etherscan.get_contract_abi(token_address) 
    contract = w3.eth.contract(token_address, abi=abi)
    return contract.functions.balanceOf(account).call() / 10**contract.functions.decimals().call()

In [81]:
sg = Subgrounds()
sg_curve_pools = sg.load_subgraph(URL_CURVE_POOLS)
sg_curve_dao = sg.load_subgraph(URL_CURVE_DAO) 

In [82]:
# frxETHCRV staked in curve gauge. frxETHCRV -> frxETHCRV-gauge at 1:1 ratio 
# ---------------------------------------------------------------------------------------------------------------------
qattrs = ['id', 'provider', 'value']
# deposits into gauge 
df_deposits = query_attrs(
    sg, sg_curve_dao.Query.gaugeDeposits(first=100000, where={'gauge': ADDRESS_frxETHCRV_GAUGE_DEPOSIT}), qattrs
)
df_deposits = remove_prefix(df_deposits, 'gaugeDeposits_') 
# withdawals from gauge 
df_withdraws = query_attrs(
    sg, sg_curve_dao.Query.gaugeWithdraws(first=100000, where={'gauge': ADDRESS_frxETHCRV_GAUGE_DEPOSIT}), qattrs
)
df_withdraws = remove_prefix(df_withdraws, 'gaugeWithdraws_') 
df_withdraws.value *= -1
# Current deposited amount per address within gauge 
df_curve = pd.concat([df_deposits, df_withdraws])
df_curve = df_curve.groupby('provider')['value'].sum().reset_index()
df_curve = df_curve.loc[df_curve.value != 0].sort_values('value', ascending=False).reset_index(drop=True)
df_curve['platform'] = 'curve-gauge'
df_curve.value /= 1e18

In [84]:
frxethCRV_curve_gauge = df_curve.value.sum()
frxethCRV_curve_gauge_actual = get_account_balance_of_erc20(ADDRESS_frxETHCRV_GAUGE_DEPOSIT, ADDRESS_frxETHCRV)
frxethCRV_convex_staked = df_curve.loc[df_curve.provider == ADDRESS_CONVEX_VOTER_PROXY].value.sum()
frxethCRV_convex_staked_actual = get_account_balance_of_erc20(ADDRESS_CONVEX_VOTER_PROXY, ADDRESS_frxETHCRV_GAUGE_DEPOSIT)

# Manual validation of frxETHCRV staked in curve gauge 
# https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a?a=0x2932a86df44fe8d2a706d8e9c5d51c24883423f5
print(f"Number of frxETHCRV deposited in curve gauge:\n\tComputed: {frxethCRV_curve_gauge:>20}\n\tActual: {frxethCRV_curve_gauge_actual:>20}")
np.testing.assert_almost_equal(frxethCRV_curve_gauge_actual, frxethCRV_curve_gauge, decimal=10)

# Manual validation of frxETHCRV-gauge (1:1 with frxETHCRV) staked in convex 
# https://etherscan.io/token/0x2932a86df44fe8d2a706d8e9c5d51c24883423f5?a=0x989aeb4d175e16225e39e87d0d97a3360524ad80
print(f"Number of frxETHCRV staked on convex:\n\tComputed: {frxethCRV_convex_staked:>20}\n\tActual: {frxethCRV_convex_staked_actual:>22}")
np.testing.assert_almost_equal(frxethCRV_convex_staked_actual, frxethCRV_convex_staked, decimal=10)

print(f"Percent of frxETHCRV staked on convex: {frxethCRV_convex_staked / frxethCRV_curve_gauge:%}")
df_curve.head()

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


Number of frxETHCRV deposited in curve gauge:
	Computed:   33200.039549120396
	Actual:     33200.0395491204
Number of frxETHCRV staked on convex:
	Computed:   30604.348806926053
	Actual:     30604.348806926053
Percent of frxETHCRV staked on convex: 92.181664%


,provider,value,platform
0,0x989aeb4d175e16225e39e87d0d97a3360524ad80,30604.348807,curve-gauge
1,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,2047.814958,curve-gauge
2,0x10e3085127c9bd92ab325f8d1f65cdcec2436149,199.879798,curve-gauge
3,0x29f227b10c58457d8837031c813216de82f9abaf,102.939595,curve-gauge
4,0x73e47e110dd251bd6449381724f2bb51c11b14bc,43.367171,curve-gauge


In [85]:
# frxETHCRV staked in convex (curve system). frxETHCRV -> cvxfrxETHCRV at 1:1 ratio 
# ---------------------------------------------------------------------------------------------------------------------
qattrs = ['id', 'amount', 'timestamp', 'user.address']
# deposits into the convex staking contract for curve lp tokens 
q_deposits = sg_curve_pools.Query.deposits(first=100000, where={'poolid_': {'swap': ADDRESS_CURVE_POOL_FRXETH_ETH}})
df_deposits = query_attrs(sg, q_deposits, qattrs)
df_deposits = remove_prefix(df_deposits, 'deposits_') 
# withdrawals from the convex staking contract for curve lp tokens 
q_withdrawals = sg_curve_pools.Query.withdrawals(first=100000, where={'poolid_': {'swap': ADDRESS_CURVE_POOL_FRXETH_ETH}})
df_withdrawals = query_attrs(sg, q_withdrawals, qattrs)
df_withdrawals = remove_prefix(df_withdrawals, 'withdrawals_') 
df_withdrawals.amount *= -1
# current deposited amount per account in convex staking contract for curve lp tokens 
df_convex = pd.concat([df_deposits, df_withdrawals])
df_convex.amount /= 1e18
df_convex['platform'] = 'convex-curve'
df_convex = df_convex.rename(columns={'user_address': 'account'})
df_convex.head()

,id,amount,timestamp,account,platform
0,0x007562ec43ffc63894d5b30e812bd65858605829090a...,1.019416,1669427219,0x0f903834187d37ff29dc2d607dd9fb50eb36b2b5,convex-curve
1,0x012a7730ff93a15624d92a94a9a8a5fa9ab8e2ff7b28...,6.79953,1668665207,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
2,0x01d17a19490fc0bbc90d695152c9d60cbd071f40fba6...,199.860807,1668970739,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
3,0x01d5801d769773d6fa0722668f1c8fcbf13f2e425c93...,10.0,1669056719,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve
4,0x03a493a4bcefca190cf3fe4c95d3564c9f262c43ec3f...,533.486109,1667486183,0x5a40ab806a038afe9c4ef413d36b6356a5e4d2c5,convex-curve


In [93]:
# Ensure that the holdings of frxETHCRV-gauge by the convex voter proxy address are the same (computed in two ways) 
# 1. Curve dao subgraph - aggregating curve gauge deposits and withdraws by address (convex voter proxy is one of them). 
# 2. Convex curve pools subgraph - aggregating convex gauge deposits and withdawals by address. 
# These numbers should be really close but sometimes might differ due to subgraph indexing speed. 
convex_frxethcrv_1 = df_curve.loc[df_curve.provider == ADDRESS_CONVEX_VOTER_PROXY]['value'].values.tolist()[0]
convex_frxethcrv_2 = df_convex.amount.sum()
print(f"Convex frxETHCRV-gauge (curve dao subgraph): {convex_frxethcrv_1}")
print(f"Convex frxETHCRV-gauge (convex curve pool subgraph): {convex_frxethcrv_2}")
np.testing.assert_almost_equal(convex_frxethcrv_1, convex_frxethcrv_2, 10)

Convex frxETHCRV-gauge (curve dao subgraph): 30604.348806926053
Convex frxETHCRV-gauge (convex curve pool subgraph): 30604.348806926042


In [87]:
df_curve.head(1)

,provider,value,platform
0,0x989aeb4d175e16225e39e87d0d97a3360524ad80,30604.348807,curve-gauge


In [88]:
df_convex.head(1)

,id,amount,timestamp,account,platform
0,0x007562ec43ffc63894d5b30e812bd65858605829090a...,1.019416,1669427219,0x0f903834187d37ff29dc2d607dd9fb50eb36b2b5,convex-curve


In [89]:
# Replace the single row in the curve gauge table with all the rows in the convex gauge table. 
df_curve_no_convex = df_curve.loc[df_curve.provider != ADDRESS_CONVEX_VOTER_PROXY]
assert len(df_curve_no_convex) + 1 == len(df_curve)
df = (
    pd.concat([
        df_curve_no_convex.rename(columns={'provider': 'account', 'value': 'amount'})[['account', 'amount', 'platform']], 
        df_convex[['account', 'amount', 'platform']]
    ])
    .groupby(['account', 'platform'])['amount'].sum().reset_index()
    .sort_values('amount', ascending=False).reset_index(drop=True) 
)
# This total should match the amount of frxETHCRV held by the curve gauge deposit 
# https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a?a=0x2932a86df44fe8d2a706d8e9c5d51c24883423f5
computed = df.amount.sum()
actual = get_account_balance_of_erc20(ADDRESS_frxETHCRV_GAUGE_DEPOSIT, ADDRESS_frxETHCRV) 
np.testing.assert_almost_equal(actual, computed, decimal=10)
df.head()

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:87: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


,account,platform,amount
0,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve,26923.951785
1,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,curve-gauge,2047.814958
2,0x3cf54f3a1969be9916dad548f3c084331c4450b5,convex-curve,1559.840998
3,0xa1175a219dac539f2291377f77afd786d20e5882,convex-curve,612.773914
4,0xdb722dd612d35a12e718138e7296e6ca3531fb98,convex-curve,327.994192


In [90]:
address_convex_staking_wrapper_frax = '0x4659d5fF63A1E1EDD6D5DD9CC315e063c95947d0'
contract_convex_staking_wrapper_frax = w3.eth.contract(
    address_convex_staking_wrapper_frax, abi=etherscan.get_contract_abi(address_convex_staking_wrapper_frax)
)

In [91]:
deposit_logs = contract_convex_staking_wrapper_frax.events.Deposited.getLogs(fromBlock=0)
withdrawal_logs = contract_convex_staking_wrapper_frax.events.Withdrawn.getLogs(fromBlock=0)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:164: DeprecationWarning: abi.decode_single() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


In [92]:
# frxETHCRV staked in convex (frax system). frxETHCRV -> stkcvxfrxETHCRV-frax
# ---------------------------------------------------------------------------------------------------------------------
deposits = [{'account': d.args['_account'], 'amount': d.args['_amount'] / 1e18} for d in deposit_logs]
withdrawals = [{'account': d.args['_user'], 'amount': -d.args['_amount'] / 1e18} for d in withdrawal_logs]

df_stkcvxfrxeth = pd.DataFrame(deposits + withdrawals).groupby('account')['amount'].sum().reset_index()
df_stkcvxfrxeth = df_stkcvxfrxeth.sort_values('amount', ascending=False).reset_index(drop=True)
df_stkcvxfrxeth = df_stkcvxfrxeth.loc[df_stkcvxfrxeth.amount != 0]

# Determine if this address represents a vault owned by some user 
abi_convex_staking_proxy = '[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"FEE_DENOMINATOR","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_rewardsAddress","type":"address"}],"name":"changeRewards","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"checkpointRewards","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"convexCurveBooster","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"convexDepositToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"crv","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"curveLpToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"cvx","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"earned","outputs":[{"internalType":"address[]","name":"token_addresses","type":"address[]"},{"internalType":"uint256[]","name":"total_earned","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"feeRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"fxs","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_claim","type":"bool"},{"internalType":"address[]","name":"_rewardTokenList","type":"address[]"}],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_claim","type":"bool"}],"name":"getReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_owner","type":"address"},{"internalType":"address","name":"_stakingAddress","type":"address"},{"internalType":"address","name":"_stakingToken","type":"address"},{"internalType":"address","name":"_rewardsAddress","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditional","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditionalConvexToken","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"_addl_liq","type":"uint256"}],"name":"lockAdditionalCurveLp","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"},{"internalType":"uint256","name":"new_ending_ts","type":"uint256"}],"name":"lockLonger","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"poolRegistry","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"rewards","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_proxy","type":"address"}],"name":"setVeFXSProxy","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLocked","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLockedConvexToken","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_liquidity","type":"uint256"},{"internalType":"uint256","name":"_secs","type":"uint256"}],"name":"stakeLockedCurveLp","outputs":[{"internalType":"bytes32","name":"kek_id","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"stakingAddress","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"stakingToken","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"usingProxy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"vaultType","outputs":[{"internalType":"enum IProxyVault.VaultType","name":"","type":"uint8"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"vaultVersion","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"vefxsProxy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"}],"name":"withdrawLocked","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_kek_id","type":"bytes32"}],"name":"withdrawLockedAndUnwrap","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
vault_owners = {}
for a in df_stkcvxfrxeth.account.unique(): 
    is_vault = True if w3.eth.getCode(a) else False
    if is_vault: 
        contract = w3.eth.contract(a, abi=abi_convex_staking_proxy)
        owner = contract.functions.owner().call()
        vault_owners[a] = owner
        
# Update table with vault info 
df_stkcvxfrxeth['vault_address'] = df_stkcvxfrxeth.account.apply(lambda a: a if a in vault_owners else None)
mask_vault = ~df_stkcvxfrxeth.vault_address.isna()
df_stkcvxfrxeth.loc[mask_vault, 'account'] = df_stkcvxfrxeth.loc[mask_vault].account.apply(lambda a: vault_owners[a])
assert all(df_stkcvxfrxeth.loc[mask_vault].account != df_stkcvxfrxeth.loc[mask_vault].vault_address)

# Add in platform information 
df_stkcvxfrxeth['platform'] = 'convex-frax-no-vault'
df_stkcvxfrxeth.loc[~df_stkcvxfrxeth.vault_address.isna(), 'platform'] = 'convex-frax-vault'
df_stkcvxfrxeth.tail()

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/web3/method.py:215: DeprecationWarning: getCode is deprecated in favor of get_code
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:87: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


,account,amount,vault_address,platform
168,0xFf6A37C2fb94C3fB47CbeAB8053722084C546a32,0.452667,0xbaaf59e5572Ba0432D33Cd7bC09C1FDAA6393e0f,convex-frax-vault
169,0x8d64898e2f89e3Ca1fcD5684882812EB4960c985,0.199905,0xB5f32EbB40b7dE0fB73E1Da427B1047c8F5C3ee0,convex-frax-vault
170,0x5180db0237291A6449DdA9ed33aD90a38787621c,0.100000,None,convex-frax-no-vault
171,0x439Adfe7CD81d8f471C16528Dbc034F87Aceb599,0.024000,0xd1a2971C3E54A476D348a073e88b45EBF2C59e81,convex-frax-vault
172,0x439Adfe7CD81d8f471C16528Dbc034F87Aceb599,0.020000,None,convex-frax-no-vault


In [94]:
# Ensure that the holdings of stkfrxETHCRV-frax by the convex frax staking wrapper are the same (computed in two ways) 
# 1. Convex curve pools subgraph - aggregating convex gauge deposits and withdawals by address. 
# 2. Event logs for convex frax staking wrapper - aggregating deposits and withdrawals by address
# These numbers should be really close but sometimes might differ due to subgraph indexing speed. 
stkcvxfrxethcrv_1 = df.loc[df.account == address_convex_staking_wrapper_frax.lower()]['amount'].values.tolist()[0]
stkcvxfrxethcrv_2 = df_stkcvxfrxeth.amount.sum()
print(f"Convex stkfrxETHCRV-frax (convex curve staking): {stkcvxfrxethcrv_1}")
print(f"Convex stkfrxETHCRV-frax (convex frax staking): {stkcvxfrxethcrv_2}")
np.testing.assert_almost_equal(stkcvxfrxethcrv_1, stkcvxfrxethcrv_2, 10)

Convex stkfrxETHCRV-frax (convex curve staking): 26923.951784666428
Convex stkfrxETHCRV-frax (convex frax staking): 26923.951784666435


In [95]:
df.head(1)

,account,platform,amount
0,0x4659d5ff63a1e1edd6d5dd9cc315e063c95947d0,convex-curve,26923.951785


In [96]:
df_stkcvxfrxeth.head(1)

,account,amount,vault_address,platform
0,0x8D8B9c79196f32161BcB2A9728D274B3b45eB9AF,4968.188152,0x1Cf851733F952c7Bc2aE375e370528d162d066e0,convex-frax-vault


In [98]:
df_no_stkcvxfrxeth = df.loc[df.account != address_convex_staking_wrapper_frax.lower()]
assert len(df) == 1 + len(df_no_stkcvxfrxeth)
df_final = (
    pd.concat([df_no_stkcvxfrxeth, df_stkcvxfrxeth[['account', 'platform', 'amount']]])
    .groupby(['account', 'platform'])['amount'].sum().reset_index()
    .sort_values('amount', ascending=False).reset_index(drop=True)
)
# This amount should match the amount held in the gauge for this curve pool. Check this value here: 
# https://etherscan.io/token/0xf43211935c781d5ca1a41d2041f397b8a7366c7a#balances
computed = df_final.amount.sum()
actual = get_account_balance_of_erc20(ADDRESS_frxETHCRV_GAUGE_DEPOSIT, ADDRESS_frxETHCRV) 
np.testing.assert_almost_equal(actual, computed, decimal=10)
df_final.head()

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:87: DeprecationWarning: abi.encode_abi() and abi.encode_abi_packed() are deprecated and will be removed in version 4.0.0 in favor of abi.encode() and abi.encode_packed(), respectively
  warnings.warn(
/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/eth_abi/codec.py:191: DeprecationWarning: abi.decode_abi() is deprecated and will be removed in version 4.0.0 in favor of abi.decode()
  warnings.warn(


,account,platform,amount
0,0x8D8B9c79196f32161BcB2A9728D274B3b45eB9AF,convex-frax-vault,4968.188152
1,0x8306300ffd616049FD7e4b0354a64Da835c1A81C,convex-frax-vault,4325.813192
2,0xD07993c6cb9692a71522Baf970A31069034dF2B0,convex-frax-vault,3170.969205
3,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,curve-gauge,2047.814958
4,0x3cf54f3a1969be9916dad548f3c084331c4450b5,convex-curve,1559.840998


In [112]:
display(HTML((
    alt.Chart(df_final.groupby('platform')['amount'].sum().reset_index())
    .transform_joinaggregate(amount_total="sum(amount)")
    .transform_calculate(fraction="datum.amount / datum.amount_total")
    .mark_arc()
    .encode(
        theta="amount:Q", 
        color="platform:N", 
        tooltip=[
            alt.Tooltip("platform:N"), 
            alt.Tooltip("amount:Q", format=",d", title="frxETHCRV"),
            alt.Tooltip("fraction:Q", format=".1%", title="Share"),
        ]
    )
).to_html()))

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


In [116]:
ddf(df_final.groupby('account')['amount'].sum().reset_index().sort_values('amount', ascending=False).reset_index(drop=True))

,account,amount
0,0x8D8B9c79196f32161BcB2A9728D274B3b45eB9AF,4968.188152
1,0x8306300ffd616049FD7e4b0354a64Da835c1A81C,4325.813192
2,0xD07993c6cb9692a71522Baf970A31069034dF2B0,3170.969205
3,0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae,2047.814958
4,0x3cf54f3a1969be9916dad548f3c084331c4450b5,1559.840998
5,0x2d018f3602c1185766BC725e3390CAab6f44AE5b,1032.556578
6,0x91746d6f9DF58B9807a5BB0e54e4EA86600c2DBa,839.959569
7,0x50664edE715e131F584D3E7EaAbd7818Bb20A068,836.568162
8,0x2dE9D6f84e28F3A347b97C6a3Db51D366E65bA4C,833.672665
9,0xa1175a219dac539f2291377f77afd786d20e5882,612.773914
